In [2]:
# ruff: noqa: F401, E402
import math
import sys
from bisect import bisect_left, bisect_right
from collections import Counter, deque
from functools import cmp_to_key, lru_cache, reduce
from heapq import heapify, heappop, heappush, heanpushpop, nsmallest
from importlib import reload
from itertools import (
    accumulate,
    chain,
    combinations,
    islice,
    pairwise,
    permutations,
    product,
    starmap,
    tee,
)
from math import comb, factorial, log, sqrt
from operator import add, and_, contains, indexOf, itemgetter, neg, or_, xor
from nprint import nprint
from typing import Callable, Iterable, List, Tuple

import matplotlib.pyplot as plt
import networkx as nx
import numpy as np

import arrays
import graphs
import lists
import mathematics
import matrix
import parsing
import search
import sequences
import sets
import strings
import trees

for m in (
    arrays,
    graphs,
    lists,
    mathematics,
    matrix,
    parsing,
    search,
    sequences,
    sets,
    strings,
    trees,
):
    reload(m)


from parsing import format_binop_expression, parse_binop_expression
from sequences import find_if
from strings import StringView, splint

print()


In [66]:
def fruit_pickup_min_time(locs: List[int], types: List[int]) -> int:
    tl = {}
    for i, loc in enumerate(locs):
        t = types[i]
        tnl, txl = tl.get(t, (loc, loc))
        tl[t] = (min(loc, tnl), max(loc, txl))
    tsort = [(0, 0)] + [tl[i] for i in sorted(tl)] + [(0, 0)]
    n = len(tsort)
    print(tsort)

    @lru_cache(None)
    def pick(i: int, lr: int):
        if i == n:
            print(f"[{i}] ===> 0")
            return 0
        pp = tsort[i - 1][lr]
        mn, mx = tsort[i]
        if mx <= pp:
            # mn ... mx , pp
            # move all the way left (=0) to mn
            print(f"[{i}] {pp} \t=> {mn} \t+{pp - mn}")
            return pp - mn + pick(i + 1, 0)
        elif pp <= mn:
            # pp, mn ... mx
            # move all the way right (=1) to mx
            print(f"[{i}] {pp} \t=> {mx} \t+{mx - pp}")
            return mx - pp + pick(i + 1, 1)
        else:
            # mn, pp, mx
            # move to mx then to mn (= 0)
            opt1 = (mx - pp) + pick(i + 1, 0)
            # move to mn then to mx (= 1)
            opt2 = (pp - mn) + pick(i + 1, 1)
            print(f"[{i}] opt1: {opt1}, opt2: {opt2}")
            if opt1 < opt2:
                print(f"[{i}] {pp} \t=> {mn}  \t+{mx - pp}+{mx - mn}")
            else:
                print(f"[{i}] {pp} \t=> {mx}  \t+{pp - mn}+{mx - mn}")
            return min(opt1, opt2) + (mx - mn)

    return min(pick(1, 0), pick(1, 1))


In [14]:
def fruit_pickup_min_time(locs: List[int], types: List[int]) -> int:
    tl = {0: (0, 0), (max(types) + 1): (0, 0)}
    for t, loc in zip(types, locs):
        tnl, txl = tl.get(t) or tl.setdefault(t, [loc, loc])
        tl[t][:] = min(loc, tnl), max(loc, txl)
    tsort = [tl[i] for i in sorted(tl)]
    pick = [0, 0]
    for i in range(len(tsort) - 1):
        mn, mx = tsort[i]
        pl, pr = pick
        for lr in (0, 1):
            np = tsort[i + 1][lr]
            if mx <= np:
                # mn ... mx <= pp
                # move all the way left (=0) to mn
                pick[lr] = np - mn + pl
            elif np <= mn:
                # pp <= mn ... mx
                # move all the way right (=1) to mx
                pick[lr] = mx - np + pr
            else:
                # mn, pp, mx
                # opt1: move to mx then to mn (= 0)
                # opt2: move to mn then to mx (= 1)
                pick[lr] = min(mx - np + pl, np - mn + pr) + (mx - mn)
    return min(pick)


In [15]:
fruit_pickup_min_time([1, 3, 5, 7], [1, 2, 3, 1])


18

In [16]:
fruit_pickup_min_time([-4, -3, 1, -8], [4, 2, 4, 5])


24

In [17]:
fruit_pickup_min_time([1, -8, -1, 2, -2, -5], [3, 3, 3, 4, 2, 1])


26